In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
from ipywidgets import widgets, VBox
from datetime import datetime
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings("ignore")

## 2. Momentum analysis

In [7]:
def eclairys_v2(stock_ticker, market_ticker, short_period=[5, 10, 20], long_period=[50, 100, 200]):
    """
    #### Description:
    Calculates trend analysis indicators (GRADE and GPS) for a specified stock and market comparison, including:
    - 'GRADE': A scoring metric derived from normalized MAD values across all periods.
    - 'Absolute GPS' and 'Market GPS': Category classifications indicating trend direction and strength 
      based on booleanized MAD values for short and long periods.
    
    #### Parameters:
    - stock_ticker (str): The stock ticker symbol for trend analysis.
    - market_ticker (str): The market index ticker symbol for relative trend analysis.
    - short_period (list): List of short periods for moving average calculation.
    - long_period (list): List of long periods for moving average calculation.

    #### Returns:
    - DataFrame: Combined DataFrame containing trend indicators for stock and market.
    """
    
    # Fetch historical price data for the stock and market
    stock_history = yf.Ticker(stock_ticker).history('max')['Close'].tz_localize(None).dropna()
    stock_history.name = stock_ticker
    market_history = yf.Ticker(market_ticker).history('max')['Close'].tz_localize(None).dropna()
    market_history.name = market_ticker
    period_list = short_period + long_period

    # Initialize DataFrames to store Moving Averages (MA), Moving Average Differentials (MAD), and normalized MAD (MADN)
    MA_df, MAD_df, MADN_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    # Calculate indicators for the stock
    for period in period_list:
        if stock_history.empty or len(stock_history) < period:  # Check if enough data exists for the period
            # If not enough data, fill with NaN
            MA_df[period] = np.nan
            MAD_df[period] = np.nan
            MADN_df[period] = np.nan
        else:
            # Calculate Moving Average (MA) and drop NaN values
            MA = stock_history.rolling(period).mean().dropna()
            MA_df[f'MA {period}'] = MA
            
            # Calculate Moving Average Differential (MAD) as the difference over 5 periods
            MAD = MA.diff(5).dropna()
            MAD_df[period] = MAD
            
            # Normalize MAD to a 0-100 scale (MADN)
            MADN = 100 * (MAD - MAD.min()) / (MAD.max() - MAD.min())
            MADN_df[period] = MADN

    # Calculate the GRADE score as the mean of MADN values across all periods
    MADN_df = MADN_df.dropna()
    GRADE = MADN_df.mean(axis=1)
    GRADE.name = 'GRADE'

    # Generate GPS indicator based on booleanized MAD values
    if GRADE.empty:
        GPS = pd.Series(dtype='object')
    else:
        # Convert MAD values to boolean values: 1 if positive, 0 if negative
        BMAD = (MAD_df >= 0).astype(int)
        
        # Sum boolean values for short and long periods separately
        SBMAD = BMAD[short_period].sum(axis=1).reindex(stock_history.index, fill_value=0)
        LBMAD = BMAD[long_period].sum(axis=1).reindex(stock_history.index, fill_value=0)

        # Define GPS conditions based on the sums of short and long period boolean MADs
        conditions = [
            (SBMAD >= 2) & (LBMAD >= 2),
            (SBMAD < 2) & (LBMAD < 2),
            (SBMAD >= 2) & (LBMAD < 2),
            (SBMAD < 2) & (LBMAD >= 2)
        ]
        choices = ['A', 'B', 'C', 'P']
        
        # Assign GPS categories based on conditions
        GPS = pd.Series(np.select(conditions, choices, default=None), index=stock_history.index, name='Absolute GPS')
        GPS = GPS.loc[GRADE.index[0]:]

    # Combine stock indicators into a DataFrame
    stock_df = pd.concat([pd.DataFrame(GRADE), pd.DataFrame(GPS), pd.DataFrame(stock_history), MA_df], axis=1)
    
    # Calculate indicators for the market comparison (relative trend between stock and market)
    relative_history = stock_history / market_history
    MA_df, MAD_df, MADN_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    
    for period in period_list:
        if relative_history.empty or len(relative_history) < period:  # Check if enough data exists for the period
            # If not enough data, fill with NaN
            MA_df[period] = np.nan
            MAD_df[period] = np.nan
            MADN_df[period] = np.nan
        else:
            # Calculate Moving Average (MA) for relative trend
            MA = relative_history.rolling(period).mean().dropna()
            MA_df[f'MA {period}'] = MA
            
            # Calculate Moving Average Differential (MAD) as the difference over 5 periods
            MAD = MA.diff(5).dropna()
            MAD_df[period] = MAD
            
            # Normalize MAD to a 0-100 scale (MADN)
            MADN = 100 * (MAD - MAD.min()) / (MAD.max() - MAD.min())
            MADN_df[period] = MADN

    # Calculate the GRADE score for the relative trend
    MADN_df = MADN_df.dropna()
    GRADE = MADN_df.mean(axis=1)
    
    # Generate Market GPS indicator based on booleanized MAD values for relative trend
    if GRADE.empty:
        market_gps = pd.Series(dtype='object')
    else:
        BMAD = (MAD_df >= 0).astype(int)
        SBMAD = BMAD[short_period].sum(axis=1).reindex(relative_history.index, fill_value=0)
        LBMAD = BMAD[long_period].sum(axis=1).reindex(relative_history.index, fill_value=0)

        # Define GPS categories based on conditions for the relative trend
        conditions = [
            (SBMAD >= 2) & (LBMAD >= 2),
            (SBMAD < 2) & (LBMAD < 2),
            (SBMAD >= 2) & (LBMAD < 2),
            (SBMAD < 2) & (LBMAD >= 2)
        ]
        choices = ['A', 'B', 'C', 'P']
        
        # Assign Market GPS categories based on conditions
        market_gps = pd.Series(np.select(conditions, choices, default=None), index=relative_history.index, name='Market GPS')
        market_gps = market_gps.loc[GRADE.index[0]:]
    
    # Combine stock and market GPS indicators into a single DataFrame
    output = pd.concat([stock_df, market_gps], axis=1)
    columns = [stock_ticker] + list(MA_df.columns) + ['GRADE', 'Absolute GPS', 'Market GPS']
    output = output.reindex(columns=columns)
    return output


In [8]:
# Exemple d'utilisation eclairys_v2
eclairys_v2('STLAP.PA', '^FCHI')

,STLAP.PA,MA 5,MA 10,MA 20,MA 50,MA 100,MA 200,GRADE,Absolute GPS,Market GPS
Date,,,,,,,,,,
2001-09-03,11.325252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-09-04,11.325252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-09-05,10.212907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-09-06,10.198140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-09-07,10.198140,10.651938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-11-07,13.040000,12.772400,12.6476,12.3715,13.03768,14.83494,18.346571,33.371753,C,C
2024-11-08,12.530000,12.734000,12.6332,12.4055,12.98544,14.76996,18.314588,32.794111,C,C
2024-11-11,12.754000,12.765200,12.6380,12.4402,12.94008,14.70376,18.282462,32.629080,C,C


In [11]:
# Widget pour entrer le ticker librement
stock_ticker_input = widgets.Text(
    description='Stock ticker:',
    placeholder='ex: AAPL',
    value='AAPL'
)

market_ticker_input = widgets.Text(
    description='Ticker:',
    placeholder='ex: ^GSPC',
    value='^GSPC'
)

# Widget pour la sélection de la date
date_picker = widgets.DatePicker(
    description='Date:',
    value=datetime(2024, 10, 31)
)

# Bouton de validation
run_button = widgets.Button(description="Valider")

# Fonction qui s'exécute lorsque le bouton est cliqué
def on_button_click(b):
    # Efface les sorties précédentes
    clear_output(wait=True)
    
    # Affiche les widgets de nouveau pour maintenir l'interface
    display(VBox([stock_ticker_input, market_ticker_input, date_picker, run_button]))
    
    # Récupère la valeur du ticker entrée par l'utilisateur
    stock_ticker = stock_ticker_input.value 
    market_ticker = market_ticker_input.value  
    # Récupère la date sélectionnée
    date = date_picker.value  
    
    # Récupération de l'historique des prix pour le ticker saisi
    stock_history = yf.Ticker(stock_ticker).history(period='max')['Close'].tz_localize(None)
    stock_history.name = stock_ticker

    market_history = yf.Ticker(market_ticker).history(period='max')['Close'].tz_localize(None)
    market_history.name = market_ticker
    
    # Exécution de la fonction eclairys avec les paramètres sélectionnés
    output = eclairys_v2(stock_ticker, market_ticker)
    
    # Filtrer les données pour la date sélectionnée
    try:
        output_date = output.loc[date]  # Utilisez la date directement comme index
        # Affichage des résultats
        print("\nRésultats Eclairys:")
        for key, value in output_date.items():
            print(f"{key}: {value}")
    except KeyError:
        print(f"No data available for the selected date: {date.strftime('%Y-%m-%d')}")
        
# Liaison du bouton avec la fonction de validation
run_button.on_click(on_button_click)

# Afficher les widgets ensemble
display(VBox([stock_ticker_input, market_ticker_input, date_picker, run_button]))



Résultats Eclairys:
AAPL: 225.66172790527344
MA 5: 230.6442443847656
MA 10: 232.06068725585936
MA 20: 230.37803802490234
MA 50: 226.9165466308594
MA 100: 222.98767120361327
MA 200: 201.48746543884278
GRADE: 54.12591015377109
Absolute GPS: P
Market GPS: A
